In [1]:
using XLSX
using CSV
using DataFrames
using StatsBase
using Distances
using LinearAlgebra
using PyCall
using PyPlot
using Random
import IterTools
std = StatsBase.std
pd = pyimport("pandas")
np = pyimport("numpy")

statistical_range(x) = maximum(x) - minimum(x)
hav(df) = begin
    data = convert(Matrix, df[:, [:LON, :LAT]])
    res = pairwise(Haversine(), data, dims=1)
    [res |> mean, res |> sum]
end

potential = DataFrame(CSV.File("Hospital Potential.csv"))

liaoning = potential[potential.Province .== "辽宁省", :]
liaoning = liaoning[.!(liaoning[!, :1] .|> ismissing), :]
sort!(liaoning, "MSD Hospital ID_1")
liaoning[!, "MSD Hospital ID_1"] = convert.(Int, liaoning[!, "MSD Hospital ID_1"])
liaoning = liaoning[:, [Symbol("MSD Hospital ID_1"),
                        Symbol("MSD Hospital Name_1"), 
                        :Province,
                        :City,
                        :County,
                        Symbol("Anti-diabetes Market Potential_MAT201909(Value, RMB)"),
                        Symbol("Sales Value_Januvia/Janumet_MAT201909(Value, RMB)"), 
                        Symbol("Market Share%_Januvia/Janumet_MAT201909")]]
rename!(liaoning, [:ID, :Name, :Province, :City, :County, :PT, :Amount, :Share]);

In [2]:
rtw = DataFrame(XLSX.readtable("DBU_Dragon RTW 2021Q1.xlsx", "RTW-2021Q1")...);
rtw = rtw[rtw[!,"医院省份"] .== "辽宁省", :]
rtw = rtw[:, [:CustomerID, :CustomerName, :TerritoryCode, :Portential, :Region]]
rename!(rtw, [:ID, :Name, :TC, :PT, :Region])
rtw.ID = convert.(Int, rtw.ID)
unique!(rtw)
sort!(rtw, :ID)
dropmissing!(rtw)

,ID,Name,TC,PT,Region
,Int64,Any,Any,Any,Any
1,900000626,东港市中心医院,DB20_24_09,1029690,DBDragon_North
2,900000636,营口经济技术开发区中心医院,DB20_24_06,887250,DBDragon_North
3,900002633,铁岭县中心医院,DB20_10_05,425995,DBDragon_North
4,900002635,抚顺市矿务局总医院,DB20_10_03,2590809,DBDragon_North
5,900002636,辽阳市中心医院,DB20_24_05,2259019,DBDragon_North
6,900004219,丹东中医院,DB20_24_09,387543,DBDragon_North
7,900004313,中国人民解放军32295部队医院,DB20_24_05,1673070,DBDragon_North
8,900004316,中国人民解放军三一三医院,DB20_24_01,648012,DBDragon_North
9,900004699,辽宁省健康产业集团阜新矿总医院,DB20_24_02,642810,DBDragon_North


In [3]:
structure = DataFrame(XLSX.readtable("Dragon structure with sales and listing.xlsx", "Sheet1")...)
structure = structure[:, [1,2,4,5,6,13,end-2]]
rename!(structure, [:ID, :Name, :County, :City, :Province, :PT, :Amount])
replace!(structure.Amount, "-"=>0)
structure = structure[structure[!,:Province] .== "辽宁省", :]
structure = structure[typeof.(structure.ID) .== Int, :]
sort!(structure, :ID)

,ID,Name,County,City,Province,PT
,Any,Any,Any,Any,Any,Any
1,900000626,东港市中心医院,东港市,丹东市,辽宁省,1029690
2,900000636,营口经济技术开发区中心医院,鲅鱼圈区,营口市,辽宁省,887250
3,900000637,辽阳第五人民医院,文圣区,辽阳市,辽宁省,139315
4,900001335,本溪满族自治县第三人民医院,本溪满族自治县,本溪市,辽宁省,81507
5,900002633,铁岭县中心医院,铁岭县,铁岭市,辽宁省,425995
6,900002635,抚顺市矿务局总医院,新抚区,抚顺市,辽宁省,2590809
7,900002636,辽阳市中心医院,文圣区,辽阳市,辽宁省,2259019
8,900002753,新宾满族自治县中医院,新宾满族自治县,抚顺市,辽宁省,120634
9,900002756,辽阳市白塔医院,白塔区,辽阳市,辽宁省,49053


In [4]:
location = DataFrame(CSV.File("Hosp_Coord_universe.csv"))
rename!(location, [:ID, :Name, :Address, :LON, :LAT])

,ID,Name,Address
,Int64,String,String
1,900020784,萍乡矿业集团有限责任公司职工总医院,江西省萍乡市芦溪县西环路226号
2,900022506,滨州市第二人民医院,山东省滨州市沾化区城富电路339号
3,900765751,滨州市滨城区市直机关医院,山东省滨州市滨城区滨州市黄河六路530号
4,902151039,博湖县妇幼保健院,新疆维吾尔自治区巴音郭楞蒙古自治州博湖县光明南路
5,900922102,晋煤集团总医院,山西省晋城市城区北石店村
6,902326023,宜宾市第三人民医院,四川省宜宾市翠屏区将军街58号
7,900767457,定南县第二人民医院,江西省赣州市定南县建设西路9号
8,900015773,荆门市中医院,湖北省荆门市东宝区荆门市象山三路5号
9,900008385,日照市东港医院,山东省日照市东港区望海路66号


In [5]:
liaoning = innerjoin(location[!,[1,4,5]], structure, on = :ID)
liaoning = liaoning[:, [:ID, :Name, :Province, :City, :County, :LAT, :LON, :PT, :Amount]]
liaoning = outerjoin(liaoning, rtw[:, [:ID, :TC, :Region]], on=:ID)
sort!(liaoning, :PT, rev=true)
unique!(liaoning)

,ID,Name,Province,City,County,LAT
,Int64,Any,Any,Any,Any,Float64?
1,900006877,铁岭市中心医院,辽宁省,铁岭市,银州区,42.2876
2,900007654,锦州市中心医院,辽宁省,锦州市,古塔区,41.1279
3,900007457,本溪市中心医院,辽宁省,本溪市,明山区,41.3071
4,900006894,抚顺市中心医院,辽宁省,抚顺市,顺城区,41.8874
5,900002635,抚顺市矿务局总医院,辽宁省,抚顺市,新抚区,41.8645
6,900015164,锦州医科大学附属第一医院,辽宁省,锦州市,古塔区,41.1397
7,900006840,朝阳市中心医院,辽宁省,朝阳市,双塔区,41.5581
8,900006691,营口市中心医院,辽宁省,营口市,西市区,40.6647
9,900006812,葫芦岛市中心医院,辽宁省,葫芦岛市,连山区,40.7684


In [6]:
updated = DataFrame(XLSX.readtable("LN_SLS_0425.xlsx","Sheet1")...)
updated = updated[!,["Hospital ID", "Hospital Name", "County", "City", "Province",
                    "Listing Flag", "Sales (2020)", "Potential", "Current TC", "Current_DSM", "Current_PSR"]]
rename!(updated, ["Hospital ID"=>"ID", "Hospital Name"=>"Name", "Potential"=>"PT"])
updated = sort(innerjoin(updated, location[!,Not(:Name)], on=:ID), :PT, rev=true)

,ID,Name,County,City,Province
,Any,Any,Any,Any,Any
1,900006877,铁岭市中心医院,银州区,铁岭市,辽宁省
2,900007654,锦州市中心医院,古塔区,锦州市,辽宁省
3,900007457,本溪市中心医院,明山区,本溪市,辽宁省
4,900006894,抚顺市中心医院,顺城区,抚顺市,辽宁省
5,900002635,抚顺市矿务局总医院,新抚区,抚顺市,辽宁省
6,900015164,锦州医科大学附属第一医院,古塔区,锦州市,辽宁省
7,900006840,朝阳市中心医院,双塔区,朝阳市,辽宁省
8,900006691,营口市中心医院,西市区,营口市,辽宁省
9,900006812,葫芦岛市中心医院,连山区,葫芦岛市,辽宁省


#### Distance matrix

In [7]:
dm = pairwise(Haversine(), Array(updated[:, [:LON, :LAT]]), dims=1)
dmf = DataFrame(dm, :auto)
idx = updated.ID .|> Symbol
rename!(dmf, idx)
# CSV.write("dm.csv", dmf)

,900006877,900007654,900007457,900006894,900002635,900015164,900006840,900006691
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,2.6193e5,1.09327e5,44654.3,47163.4,2.60641e5,294608.0,2.25983e5
2,2.6193e5,0.0,2.22525e5,247227.0,2.4606e5,1500.71,74035.6,1.06852e5
3,1.09327e5,2.22525e5,0.0,65577.7,63009.3,2.2167e5,2.78722e5,1.47393e5
4,44654.3,247227.0,65577.7,0.0,2570.14,2.46084e5,290074.0,1.95232e5
5,47163.4,2.4606e5,63009.3,2570.14,0.0,2.44926e5,2.89449e5,1.93219e5
6,2.60641e5,1500.71,2.2167e5,2.46084e5,2.44926e5,0.0,73748.1,1.06852e5
7,294608.0,74035.6,2.78722e5,290074.0,2.89449e5,73748.1,0.0,1.79927e5
8,2.25983e5,1.06852e5,1.47393e5,1.95232e5,1.93219e5,1.06852e5,1.79927e5,0.0
9,3.01565e5,45337.1,2.5085e5,283067.0,2.81665e5,46837.7,94656.6,1.15911e5


In [8]:
# dm = rename(DataFrame(CSV.File("vincenty.csv")), "Column1"=>"ID")
# idx = ∈(updated.ID).(dm.ID)
# dm = dm[idx, :][!, 2:end][!, idx] |> Array

#### Potential vector

In [9]:
grouped_liaoning = groupby(sort(updated|>dropmissing, "Sales (2020)", rev=true), "Current TC")

immutable_hospitals = []
for i ∈ grouped_liaoning
    threshold = sum(i."Sales (2020)")/2
    idx = cumsum(i."Sales (2020)") .≥ threshold
    immutable_hospital = idx[1] == 1 ? i.ID[1] : i.ID[1:sum(.!idx)+1]
    highest_potential_hospital = i.ID[sortperm(i.PT, rev=true)[1]]
    immutable_hospital = [immutable_hospital; highest_potential_hospital]
    append!(immutable_hospitals, immutable_hospital)
end

unique!(immutable_hospitals)

15-element Vector{Any}:
 900010106
 900006840
 900015164
 900007654
 900007653
 900007457
 900006877
 900006691
 900000636
 900002636
 900045316
 900013543
 900000626
 900002635
 900006894

In [10]:
obj(partition, df_t, n_territories) = begin

    territorial_distances =  [dm[partition .== i, partition .== i] |> mean for i ∈ 1:n_territories]
#    territorial_distances = map(distance, [df_t[partition .== i, :] for i ∈ 1:n_territories])
    sum_territorial_distance = territorial_distances |> sum
    mean_territoial_distance = territorial_distances |> mean
    range_territorial_distance = territorial_distances |> statistical_range
    
    territorial_potential = [df_t.PT[partition .== i] |> sum for i ∈ 1:n_territories]
    sum_territorial_potential =  territorial_potential|> sum
    range_territorial_potential = territorial_potential |> statistical_range
    
    [sum_territorial_distance, sum_territorial_potential, mean_territoial_distance, range_territorial_distance, range_territorial_potential]
    end

find_territory(df_t, potential) = begin
    n_territories = length(unique(df_t.TC)) - 1
    n = nrow(df_t)
    
    partition = repeat(collect(1:3), 11)
    append!(partition, repeat(collect(4:8), 10))
    partition = [partition; zeros(Int, n - length(partition))]

    ϵ = obj(partition, df_t, n_territories)
    total_no_of_improvment = 0
    
    for t ∈ 1:10
        no_of_improvment = 0
        for loc1 ∈ 1:n
            for loc2 ∈ loc1:n

                a1 = partition[loc1]; a2 = partition[loc2]
                partition[loc1] = a2; partition[loc2] = a1

                ϵ′ = obj(partition, df_t, n_territories)
                
                selected_hospitals = df_t.ID[partition .≠ 0]
                condition_on_immutables = immutable_hospitals ⊆ selected_hospitals
                
                if ϵ′[1] < ϵ[1] && ϵ′[2] >= potential && condition_on_immutables
                    ϵ = ϵ′
                    no_of_improvment = no_of_improvment + 1
                else
                    partition[loc1] = a1; partition[loc2] = a2
                end
            end
        end
        total_no_of_improvment = total_no_of_improvment + no_of_improvment
        println("Number of improvments: $(total_no_of_improvment)")
        println("Best objective value $(ϵ)")
        no_of_improvment == 0 && break
    end
    println("Total Number of improvments: $(total_no_of_improvment)")
    println("################################################")
    return partition
end;

In [11]:
updated[1:83, :].PT|>sum

8.42802986e7

In [12]:
df_ih = updated[:, :]
rename!(df_ih, "Current TC"=>"TC")
#partition = [find_territory(df_ih, i) for i ∈ range(82422773, stop = df_ih[1:83, :].PT|>sum, length=30)];
partition = find_territory(df_ih, range(82422773, stop = df_ih[1:83, :].PT|>sum, length=30)[end-7])

Number of improvments: 139
Best objective value [584322.374092508, 8.38320896e7, 73040.2967615635, 72735.86860771137, 1.45352484e7]
Number of improvments: 167
Best objective value [447751.53620781924, 8.38375566e7, 55968.942025977405, 51664.042350823845, 8.4760802e6]
Number of improvments: 194
Best objective value [374994.5878462967, 8.38448076e7, 46874.32348078709, 63498.97663179436, 8.6481522e6]
Number of improvments: 198
Best objective value [374251.2439324183, 8.38448076e7, 46781.40549155229, 58297.78814896614, 8.2684282e6]
Number of improvments: 200
Best objective value [373857.08365115477, 8.38448076e7, 46732.135456394346, 58110.75935314274, 7.3174464e6]
Number of improvments: 200
Best objective value [373857.08365115477, 8.38448076e7, 46732.135456394346, 58110.75935314274, 7.3174464e6]
Total Number of improvments: 200
################################################


515-element Vector{Int64}:
 5
 7
 2
 4
 4
 6
 1
 3
 7
 2
 2
 1
 3
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [13]:
Describe(res) = begin
    res = res[:, Not(1)]
    μ = map(mean, eachcol(res))
    median = map(StatsBase.median, eachcol(res))
    σ = map(std, eachcol(res))
    DataFrame(Symbol = names(res), Mean = μ, Median = median, Std = σ)
end

territorial_statistics(df_t) = begin
    grouped = groupby(df_t, :Cluster)
    res = combine(grouped, :PT => sum => :PT)
    res[!, :Distance] = [dm[df_t.Cluster .== i, df_t.Cluster .== i] |> mean for i ∈ res.Cluster]
    res[!, Symbol("PT/Distance")] = res.PT ./ res.Distance
    sort!(res, :PT, rev=true)
    res = res[Not(1), :]
    [res, res|>Describe]
end

territorial_statistics (generic function with 1 method)

In [14]:
df_ih[!,:Cluster] = partition;
grouped_df_ih = groupby(df_ih, :Cluster)
disruption = length(setdiff(dropmissing(df_ih).ID, df_ih[df_ih.Cluster .≠ 0, :].ID))
res = combine(grouped_df_ih, "Sales (2020)"=>sum=>"Sales (2020)", :PT=>sum=>:PT)
res[!, :Distance] = [dm[df_ih.Cluster .== i, df_ih.Cluster .== i] |> mean for i ∈ res.Cluster]
res = res[res.Cluster .≠ 0, :]
# stats = mapcols(statistical_range, res[!, Not(:Cluster)])
# rename!(stats, [Symbol("Amount Range"), Symbol("PT Range"), Symbol("")])

,Cluster,Sales (2020),PT,Distance
,Int64,Float64,Float64,Float64
1,1,1.41829e6,1.08692e7,60741.3
2,2,8.76316e5,1.40478e7,24590.6
3,3,884454.0,1.15548e7,80637.5
4,4,3.45667e5,1.03289e7,30869.4
5,5,4.65784e5,9.12732e6,22526.7
6,6,6.32208e5,8.20995e6,47108.9
7,7,3.44911e5,1.29765e7,27753.5
8,8,275378.0,6.73035e6,79629.3


In [15]:
sort!(df_ih[df_ih.Cluster.≠0, :], :Cluster)

,ID,Name,County,City,Province
,Any,Any,Any,Any,Any
1,900006840,朝阳市中心医院,双塔区,朝阳市,辽宁省
2,900010106,朝阳市第二医院,双塔区,朝阳市,辽宁省
3,901843818,喀左县第四人民医院,喀喇沁左翼蒙古族自治县,朝阳市,辽宁省
4,900009977,凌源市中心医院,凌源市,朝阳市,辽宁省
5,900031798,绥中县医院,绥中县,葫芦岛市,辽宁省
6,900005411,辽宁省建平县医院,建平县,朝阳市,辽宁省
7,900012688,凌源市中医院,凌源市,朝阳市,辽宁省
8,900009256,朝阳县人民医院,朝阳县,朝阳市,辽宁省
9,900008420,凌源市监狱管理分局中心医院,凌源市,朝阳市,辽宁省


In [16]:
CSV.write("Optimized with Immutables.csv", df_ih)

"Optimized with Immutables.csv"